<a href="https://colab.research.google.com/github/hyeminyu828/SOCAR/blob/main/HW3_week3_both.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tada_eta = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/tada_eta.xlsx')
print(tada_eta.columns)
tada_eta.head()

Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0


In [ ]:
# Ordinal Encoding
import sklearn
from sklearn.preprocessing import OrdinalEncoder

enc = sklearn.preprocessing.OrdinalEncoder(dtype=np.int32)
ordinal = enc.fit_transform(np.asarray(tada_eta['pickup_gu']).reshape(-1,1))
tada_eta['pickup_gu'] = ordinal[:,0]
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*100000
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'driver_id', 'pickup_lng', 'pickup_lat', 'driver_lng','driver_lat'],1)
tada_eta.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


,ATA,api_eta,month,pickup_gu,hour,distance
0,10.60,5.65,12,8,0,52.400202
1,9.20,8.60,12,2,0,44.465889
2,10.85,10.53,12,7,0,149.731535
3,5.52,1.92,7,23,0,1.000535
4,8.60,7.10,7,23,0,35.014797


In [ ]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)
train = tada_eta[:12000]
test = tada_eta[12000:]

x_train = np.asarray(train.drop('ATA',1)) 
x_train = np.asarray(train.drop('pickup_gu',1))
y_train = np.asarray(train['ATA'])
x_test = np.asarray(test.drop('ATA',1))
x_test = np.asarray(test.drop('pickup_gu',1))
y_test = np.asarray(test['ATA'])
eta_features = [x for i,x in enumerate(tada_eta.columns) if i!=0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


In [ ]:
# PCA on the train dataset 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# each feature should be centered (zero mean) and with unit variance
X_train_normalized = StandardScaler().fit(x_train).transform(x_train)  
train_pca = PCA(n_components = 2).fit(X_train_normalized)
X_train_pca = train_pca.transform(X_train_normalized)


In [ ]:
# PCA on the test dataset 

# each feature should be centered (zero mean) and with unit variance
X_test_normalized = StandardScaler().fit(x_test).transform(x_test)  
test_pca = PCA(n_components = 2).fit(X_test_normalized)
X_test_pca = test_pca.transform(X_test_normalized)


In [ ]:
print(X_train_pca.shape)
print(X_test_pca.shape)
print(train['pickup_gu'].shape)
print(test['pickup_gu'].shape)
X_train_pca = np.column_stack((X_train_pca, train['pickup_gu']))
X_test_pca = np.column_stack((X_test_pca, test['pickup_gu']))


(12000, 2)
(1916, 2)
(12000,)
(1916,)


In [ ]:
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

X = np.concatenate((X_train_pca,X_test_pca), axis=0)
y = np.concatenate((y_train,y_test), axis=0)

pds = PredefinedSplit(test_fold = [-1]*len(x_train)+ [0]*len(x_test))

In [ ]:
param_grid = {'max_iter': [100, 200, 300],
              'max_depth': [2, 3, 4, 5],
              'min_samples_leaf': [1, 20, 100],
              'learning_rate': [0.01, 0.02, 0.05],
              'loss': ['squared_error']}

grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid, 
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error',
                           verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)

Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.02, 'loss': 'squared_error', 'max_depth': 2, 'max_iter': 200, 'min_samples_leaf': 1}


In [ ]:
from sklearn.metrics import mean_squared_error

reg = ensemble.HistGradientBoostingRegressor(
           max_iter=200, 
           max_depth=2, 
           min_samples_leaf=1,
           learning_rate=0.02,
           categorical_features=[2],
           loss='squared_error')
reg.fit(X_train_pca, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test_pca))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_test, X_test_pca[:,0]) ))

The mean squared error (MSE) on test set: 3.1701
The initial error of API ETA on test set: 80.1504
